In [1]:
import os
import pandas as pd
import math
import numpy as np
from itertools import combinations

import csv
import datetime

In [2]:
directory = os.path.abspath("finance_data/data/")
ticket_names = []
ticket_hash = {}

In [3]:
for file in os.listdir(directory):
    if file.endswith('.csv') and 'monday' not in file:
        ticket_name = os.path.splitext(file)[0]
        ticket_names.append(ticket_name)
        df = pd.read_csv(directory + "/" + file)
        
        # we want to calculate q and r for each ticket
        q = (df.sort_values(by=['Date']).Close.diff() / df.sort_values(by=['Date']).Close.shift()).dropna()
        r = q.apply(lambda x: math.log(1+x))
        temp = {}
        temp['q'] = q
        temp['r'] = r
        ticket_hash[ticket_name] = temp
    
ticket_names = [os.path.splitext(filename)[0] for filename in os.listdir(directory) if filename.endswith('.csv')]
# ticket_combinations = [comb for comb in combinations(ticket_names, 2)]


with open(directory + "/correlation.txt", "w") as correlation_file:
    for i in range(len(ticket_names)):
        for j in range(i+1, len(ticket_names)):
            r_i = ticket_hash[ticket_names[i]]['r'] #get value from this ticket
            r_j = ticket_hash[ticket_names[j]]['r'] # get value from this ticket
            
            #TODO: need to look into this pair (KHC, PYPL)
            # for now.. let's ignore it because it's causing weird issue.
            if ticket_names[i] == 'KHC' and ticket_names[j] == 'PYPL':
                continue
            
            if len(r_i) == len(r_j): #only looking at same length for both tickets
                n = len(r_i)
                numerator = np.mean(np.multiply(r_i,r_j)) - np.mean(r_i)*np.mean(r_j)
                denomiator = np.sqrt( (np.mean( np.square(r_i) ) - np.mean(r_i)**2) * (np.mean( np.square ( r_j ) ) - np.mean(r_j) **2) )
                ro_i_j = numerator / denomiator
                w_i_j = math.sqrt(2*(1-ro_i_j))
                correlation_file.write('{} {} {}\n'.format(ticket_names[i], ticket_names[j], w_i_j))
                

In [4]:
ticket_names_2 = ticket_names

In [5]:
for file in os.listdir(directory):
    if file.endswith('.csv'):
        with open(directory + "/" + file, 'r') as inp, open(directory + "/monday_" + file, 'w') as out:
            writer = csv.writer(out)
            for row in csv.reader(inp):
                date = row[0]
                if(date == 'Date'):
                    writer.writerow(row)
                    continue
                if('-' in date):
                    split_date = date.split('-')
                elif('/' in date):
                    split_date = date.split('/')
                else:
                    continue
                if(len(split_date[0]) != 4):
                    day = datetime.datetime(int('20' + split_date[2]), int(split_date[0]), int(split_date[1]))
                else:
                    day = datetime.datetime(int(split_date[0]), int(split_date[1]), int(split_date[2]))
                if(day.weekday() == 0):
                    writer.writerow(row)

In [6]:
for file in os.listdir(directory):
    if file.endswith('.csv') and 'monday' in file:
        ticket_name = os.path.splitext(file)[0]
        ticket_names.append(ticket_name)
        df = pd.read_csv(directory + "/" + file)
        
        # we want to calculate q and r for each ticket
        q = (df.sort_values(by=['Date']).Close.diff() / df.sort_values(by=['Date']).Close.shift()).dropna()
        r = q.apply(lambda x: math.log(1+x))
        temp = {}
        temp['q'] = q
        temp['r'] = r
        ticket_hash[ticket_name] = temp
    
ticket_names = [os.path.splitext(filename)[0] for filename in os.listdir(directory) if filename.endswith('.csv') and 'monday' in filename]
# ticket_combinations = [comb for comb in combinations(ticket_names, 2)]


with open(directory + "/correlation_monday.txt", "w") as correlation_file:
    for i in range(len(ticket_names)):
        for j in range(i+1, len(ticket_names)):
            r_i = ticket_hash[ticket_names[i]]['r'] #get value from this ticket
            r_j = ticket_hash[ticket_names[j]]['r'] # get value from this ticket
            
            #TODO: need to look into this pair (KHC, PYPL)
            # for now.. let's ignore it because it's causing weird issue.
            if ticket_names[i] == 'monday_KHC' and ticket_names[j] == 'monday_PYPL':
                continue
            
            if len(r_i) == len(r_j): #only looking at same length for both tickets
                n = len(r_i)
                numerator = np.mean(np.multiply(r_i,r_j)) - np.mean(r_i)*np.mean(r_j)
                denomiator = np.sqrt( (np.mean( np.square(r_i) ) - np.mean(r_i)**2) * (np.mean( np.square ( r_j ) ) - np.mean(r_j) **2) )
                ro_i_j = numerator / denomiator
                w_i_j = math.sqrt(2*(1-ro_i_j))
                correlation_file.write('{} {} {}\n'.format(ticket_names_2[i], ticket_names_2[j], w_i_j))